In [14]:
from syft import GammaTensor, lazyrepeatarray, DataSubjectArray, PhiTensor, nn
from typing import Dict
import numpy as np
from jax.numpy import DeviceArray, add, array
from time import time

In [2]:
def _add(state: dict) -> DeviceArray:
    return add(self.run(state), other.run(state))

def no_op(x: Dict[str, GammaTensor]) -> Dict[str, GammaTensor]:
    """A Private input will be initialized with this function.
    Whenever you manipulate a private input (i.e. add it to another private tensor),
    the result will have a different function. Thus we can check to see if the f
    """
    return x

In [5]:
# This is a sample input for the backward pass
bp_input = GammaTensor(
    child=array([[0.75, 0.25],[0.75, 0.25]]), 
    data_subjects=np.array([[DataSubjectArray('22545'), DataSubjectArray('22545')],[DataSubjectArray('22545'), DataSubjectArray('22545')]],dtype=object), 
    min_vals=lazyrepeatarray(0,(2, 2)), 
    max_vals=lazyrepeatarray(1,(2, 2)), 
    is_linear=True, 
    func=_add, 
    id='611477384', 
    state={
        '726317529': GammaTensor(
            child=array([[0.75, 0.25],[0.75, 0.25]]),
            data_subjects=np.array([[DataSubjectArray('22545'), DataSubjectArray('22545')],[DataSubjectArray('22545'), DataSubjectArray('22545')]],dtype=object), 
            min_vals=lazyrepeatarray(0,(2, 2)), 
            max_vals=lazyrepeatarray(1,(2, 2)), 
            is_linear=True, 
            func=no_op, 
            id='726317529', 
            state={})
    }
)

In [7]:
model = nn.Model()

# Layer 1
model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=(2247, 3, 50, 50)))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# # Layer 2
model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 3
model.add(nn.Convolution(nb_filter=128, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 4
model.add(nn.Convolution(nb_filter=256, filter_size=3, padding=2))
model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 5
# model.add(nn.Convolution(nb_filter=512, filter_size=3, padding=2))
# model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
# model.add(nn.MaxPool(pool_size=2, stride=2))

# Layer 6
model.add(nn.AvgPool(5))

# # Layer 7
model.add(nn.Flatten())

# # Layer 8
model.add(nn.Linear(2))

model.compile()
print("Model has compiled")

Model has compiled


In [10]:
y_true = PhiTensor(
    child=np.array([0, 0]), 
    data_subjects=np.array([DataSubjectArray('22545'), DataSubjectArray('22545')], dtype=object),
    min_vals=lazyrepeatarray(0, (2,)),
    max_vals=lazyrepeatarray(1, (2,))
)

In [11]:
next_grad = model.loss.backward(bp_input, y_true)

In [12]:
next_grad

GammaTensor(child=DeviceArray([[4.        , 1.33333333],
             [4.        , 1.33333333]], dtype=float64), data_subjects=array([[DataSubjectArray: {'5', '2', '4'}, DataSubjectArray: {'5', '2', '4'}],
       [DataSubjectArray: {'5', '2', '4'}, DataSubjectArray: {'5', '2', '4'}]],
      dtype=object), min_vals=<lazyrepeatarray data: -5.33333333328 -> shape: (2, 2)>, max_vals=<lazyrepeatarray data: 5.33333333328 -> shape: (2, 2)>, is_linear=True, func=<function GammaTensor.__mul__.<locals>._mul at 0x7f5100582e50>, id='1618270190', state={'40288861': GammaTensor(child=DeviceArray([[0.75, 0.25],
             [0.75, 0.25]], dtype=float64), data_subjects=array([[DataSubjectArray: {'5', '2', '4'}, DataSubjectArray: {'5', '2', '4'}],
       [DataSubjectArray: {'5', '2', '4'}, DataSubjectArray: {'5', '2', '4'}]],
      dtype=object), min_vals=<lazyrepeatarray data: -0.99999999999 -> shape: (2, 2)>, max_vals=<lazyrepeatarray data: 0.99999999999 -> shape: (2, 2)>, is_linear=True, func=<funct

In [15]:
total_time_backward = 0.0
for layer in model.layers[::-1]:
    print(layer)
    start = time()
    next_grad = layer.backward(next_grad)
    end = time()
    print(f"Time to execute backward for {layer}: {end-start}")
    total_time_backward += end-start

Linear


AttributeError: 'NoneType' object has no attribute 'T'

### Debugging